In [1]:
import xml.etree.ElementTree as ET
import os
import numpy as np
import subprocess
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange
from collections import defaultdict
from enum import Enum
from numpy.random import choice
from random import sample

In [2]:
ls ../benchmarks/blif/

sparcT1_core_stratixiv_arch_timing.blif
sparcT1_core_stratixiv_arch_timing_gnl_0.45.blif
sparcT1_core_stratixiv_arch_timing_gnl_0.50.blif
sparcT1_core_stratixiv_arch_timing_gnl_0.55.blif
sparcT1_core_stratixiv_arch_timing_gnl_0.60.blif


In [3]:
bm_folder = '../benchmarks/'
file_name = 'sparcT1_core_stratixiv_arch_timing'
path_blif = os.path.join(bm_folder, 'blif',file_name +'.blif')
path_graphfiles_folder = os.path.join(bm_folder,'graphfile' ,file_name)
arch_name = 'stratixiv_arch.timing.xml'
path_arch_folder = os.path.join(bm_folder,'architecture' ,arch_name)
path_log_file = os.path.join(bm_folder, 'misc','vpr_stdout' +'.log')
hmetis = 'hmetis-1.5-linux/hmetis'
if not os.path.isdir(path_graphfiles_folder):
    os.mkdir(path_graphfiles_folder)

tree = ET.parse(path_arch_folder)
root = tree.getroot()

In [4]:
### Read original blif
file1 = open(path_blif, 'r')
lines = file1.readlines()
file1.close()

models = []
index1 = 0
for index2, line in enumerate(lines):
    if '.model' in line:
        models.append(lines[index1:index2])
        index1 = index2
models.append(lines[index1:])
main_model = models[1]
subckt_models = models[2:]

In [5]:
class Block:
    def __init__(self, type_, sub_type, ins, clks, outs):
        self.sub_type = sub_type
        self.type = type_
        self.inputs = ins  #port:net
        self.outputs = outs
        self.clocks = clks
        
    def add_input(self, port, net):
        self.inputs[port] = net
        
    def add_output(self, port, net):
        self.outputs[port] = net
    
    def add_clk_port(self, port):
        self.clocks.append(port)
        
        
class Netlist:
    def __init__(self):
        self.blocks = []
        self.nets = None
        
    def add_block(self, block):
        self.blocks.append(block)
        
    def net_graph(self):
        self.nets = defaultdict(lambda: defaultdict(list))
        
        for block_num, block in enumerate(self.blocks):
            subckt_type = block.type
#             if subckt_type != 'dffeas':
            for port, input_ in block.inputs.items():
                self.nets[input_]['sinks'].append({block_num: port})
            for port, output_ in block.outputs.items():
                self.nets[output_]['source'].append({block_num: port})
                    
    def get_net_graph(self):
        if not self.nets:
            self.net_graph()
        return self.nets
    
    def get_block_graph(self):
        return self.blocks
    
    def get_blocks_with_sub_type(self, sub_type):
        return [block for block in self.blocks if block.sub_type == sub_type]
    
    
    
class Subckt:
    def __init__(self, type_, inputs, outputs):
        self.data_inputs = []
        self.ctrl_inputs = []
        self.inputs = inputs
        self.outputs = outputs
        self.type = type_

 
class PortLabel(Enum):
    GNL = 1
    CTRL = 2
    CARRYCHAIN = 3
    CONSTANT = 4
    
class SubcktLabel(Enum):
    RAM = 1
    LAB = 2
    MAC = 3

class Subckt_instance:
    def __init__(self, type_, in_ports, clocks, out_ports):
        self.type = type_   
        self.in_ports = dict(zip(in_ports, len(in_ports) * [PortLabel.GNL]))
        self.out_ports = dict(zip(out_ports, len(out_ports) * [PortLabel.GNL]))
        self.clocks = clocks
        if 'ram' in self.type:
            self.label = SubcktLabel.RAM
        elif 'mac' in self.type:
            self.label = SubcktLabel.MAC
        else:
            self.label = SubcktLabel.LAB
    
    
class Subckt_instances:
    def __init__(self):
        self.subckts = {}
        self.count = defaultdict(int)
        
    def add_subckt(self, subckt):
        name = self.identify_subckt(subckt)
        if name == None:
            name = f'm{len(self.subckts):03}'
            self.subckts[name] = subckt
        self.count[name] = self.count[name] + 1
        return name
        
    def identify_subckt(self, subckt):
        for name, subckt_ in self.subckts.items():
            if subckt_.type == subckt.type:
                if subckt_.in_ports == subckt.in_ports:
                    if subckt_.out_ports == subckt.out_ports:
                        if subckt_.clocks == subckt.clocks:
                            return name
        return None
    
    def get_instances_with_label(self, label):
        return {sub_type: subckt for sub_type, subckt in self.subckts.items() if subckt.label == label}

class NetGraph:

        def __init__(self, vertices):
            # No. of vertices
            self.V = len(vertices)
            self.input_label = len(vertices) * [False]
            self.output_label = len(vertices) * [False]
            
            # default dictionary to store graph
            self.graph = defaultdict(list)

            self.Time = 0
            
            self.vertice_to_alias = dict(zip(list(vertices), np.arange(len(vertices))))
            self.alias_to_vertice = dict(zip(np.arange(len(vertices)), list(vertices)))
            assert self.vertice_to_alias[self.alias_to_vertice[100]] == 100
        
        def set_input(self, u):
            self.input_label[self.vertice_to_alias[u]] = True
            
        def set_output(self, u):
            self.output_label[self.vertice_to_alias[u]] = True
            
        def get_inputs(self):
            
            return [self.alias_to_vertice[v] for v, inp_label in zip(self.graph, self.input_label) if inp_label]
            
            
        # function to add an edge to graph
        def addEdge(self, u, v):
            self.graph[self.vertice_to_alias[u]].append(self.vertice_to_alias[v])
            
        
        # Prints all not yet visited vertices reachable from s
        
        def DFS_call(self, s):
            s = self.vertice_to_alias[s]
            self.DFS(s)
            
            
        def DFS(self, s, depth=0):           # prints all vertices in DFS manner from a given source.
                                    # Initially mark all vertices as not visited
            print(depth)
            visited = [False for i in range(self.V)]
            
            # Create a stack for DFS
            stack = []
            stack_depth = []
            # Push the current source node.
            stack.append(s)
            stack_depth.append(depth)
            # Descendants
            descendants = []
            
            while (len(stack)):
                # Pop a vertex from stack and print it
                s = stack[-1]
                depth = stack_depth[-1]
                print(depth)
                
                stack.pop()
                stack_depth.pop()
                
                # Stack may contain same vertex twice. So
                # We need to print the popped item only
                # If it is not visited

                if (not visited[s]):
                    depth+=1
                    descendants.append(s)
                    stack_depth.append(depth)
                    visited[s] = True

                # Get all adjacent vertices of the popped vertex s
                # If a adjacent has not been visited, then push it
                # to the stack.
                for node in self.graph[s]:
                    if (not visited[node]):
                        stack.append(node)
                        stack_depth.append(depth)
            return descendants
        
        def get_descendants(self, u):
            visited = [False] * (self.V)
            u = self.vertice_to_alias[u]
            descendants = self.DFS(u)
            descendants.remove(u)
            return [self.alias_to_vertice[v] for v in descendants]
            
        def DFSUtil(self, temp, v, visited):
 
            # Mark the current vertex as visited
            visited[v] = True
        
            # Store the vertex to list
            temp.append(v)

            # Repeat for all vertices adjacent
            # to this vertex v
            for i in self.graph[v]:
                if visited[i] == False:
                    # Update the list
                    temp = self.DFSUtil(temp, i, visited)
            return temp
        
        # Method to retrieve connected components
        # In an undirected graph
        def connectedComponents(self):
            visited = []
            cc = []
            for i in range(self.V):
                visited.append(False)
            for v in range(self.V):
                if visited[v] == False:
                    temp = []
                    cc.append(self.DFSUtil(temp, v, visited))
            connected_components = [[self.alias_to_vertice[v] for v in cc_] for cc_ in cc]
            connected_components_input_labels = [[self.input_label[v] for v in cc_] for cc_ in cc]
            connected_components_output_labels = [[self.output_label[v] for v in cc_] for cc_ in cc]
            return connected_components, connected_components_input_labels, connected_components_output_labels
        
        
            

In [6]:
arch_subckts = {}

for subckt_model in subckt_models:
    input_line, output_line = False, False
    inputs = []
    outputs = []
    for line in subckt_model:
        if '.model' in line:
            name = line.split()[1]
        if '.inputs' in line:
            input_line = True
            output_line = False
        elif '.outputs' in line:
            input_line = False
            output_line = True 
        elif line[0] == '.':
            input_line = False
            output_line = False
        elif input_line:
            inputs.append(line.split()[0])
        elif output_line:
            outputs.append(line.split()[0])
    
    arch_subckts[name] = Subckt(name, inputs, outputs)
    
    
    
for subckt in arch_subckts.values():
    for input_ in subckt.inputs:
        if 'data' in input_:
            subckt.data_inputs.append(input_)
        else:
            subckt.ctrl_inputs.append(input_)
    
            
arch_subckts['dffeas'].ctrl_inputs.remove('d')
arch_subckts['dffeas'].ctrl_inputs.remove('ena')
arch_subckts['dffeas'].data_inputs.append('d')
arch_subckts['dffeas'].data_inputs.append('ena')

### Collect data

In [7]:
new_main_model = []
new_line_split = []

# Format
for line in main_model:
    line_split = line.split()
    if line_split != [] and (line_split[-1] == '\\'):
        new_line_split.extend(line_split[:-1])
    else:
        new_line_split.extend(line_split)
        new_line = ' '.join(new_line_split)
        new_main_model.append(new_line_split)
        new_line_split = []
        
# collect .inputs, .outputs, .names, .latch, .subckt
inputs_main = None
outputs_main = None
names_main = []
latches_main = []
subckts_main = []

for split_line in new_main_model:
    if split_line == []:
        pass
    elif split_line[0] == '.inputs':
        inputs_main = split_line
    elif split_line[0] == '.outputs':
        outputs_main = split_line
    elif split_line[0] == '.names': 
        names_main.append(split_line)
    elif split_line[0] == '.latch':
        latches_main.append(split_line)
    elif split_line[0] == '.subckt':
        subckts_main.append(split_line)

In [8]:
netlist = Netlist()
subckt_instances = Subckt_instances()

for subckt in subckts_main:
        #subckt first: '.subckt', second: 'type_', then 'IO_name=netname'
        type_ = subckt[1]
        ins = {}
        clks = []
        outs = {}
        for io_net in subckt[2:]:
            port, net = io_net.split('=')
            if port in arch_subckts[type_].inputs:
                if 'clk' in port:
                    clks.append(port)
                else:
                    ins[port] = net
            elif port in arch_subckts[type_].outputs:
                outs[port]= net
        
        input_ports = list(ins)
        output_ports = list(outs)
        
        subckt_instance = Subckt_instance(type_, input_ports, clks, output_ports)
        instance_type = subckt_instances.add_subckt(subckt_instance)
        
        netlist.add_block(Block(type_, instance_type, ins, clks, outs))
        

## Collect data from netlist

### Logic Depth distribution

Find all logic clusters

In [9]:
blocks_comb_subset = []
block_in_comb_subset = defaultdict(bool)
for block in netlist.get_block_graph():
    if block.type != 'dffeas':
        blocks_comb_subset.append(block)
        block_in_comb_subset[block] = True

In [10]:
len(blocks_comb_subset)

46739

In [11]:
net_graph_undirectional = NetGraph(blocks_comb_subset)
net_graph_directional = NetGraph(blocks_comb_subset)

net_blocks = netlist.get_net_graph()
blocks = netlist.get_block_graph()
for source_sinks in net_blocks.values():
    block_num = source_sinks['source']
    if block_num != []:
        source_num = list(block_num[0])[0]
        source = blocks[source_num]
        if block_in_comb_subset[source]:
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]
                if block_in_comb_subset[sink]:
                    net_graph_undirectional.addEdge(source, sink)
                else:
                    net_graph_undirectional.set_output(source)
        else: # label sinks inputs
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]
                if block_in_comb_subset[sink]:
                    net_graph_undirectional.set_input(sink)

In [12]:
ccs_blocks, ccs_input_labels, ccs_output_labels = net_graph_undirectional.connectedComponents()

In [13]:
input_blocks = net_graph_undirectional.get_inputs()
input_blocks[0]
net_graph_undirectional.DFS_call(input_blocks[3])

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
28
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [14]:
net_graph_undirectional.graph[net_graph_undirectional.input_label]

TypeError: unhashable type: 'list'

In [15]:
[len(cc_blocks) for cc_blocks in ccs_blocks]

[273,
 1,
 1,
 108,
 5,
 2785,
 6,
 1,
 13,
 3903,
 13,
 20,
 23,
 532,
 567,
 3457,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 127,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 4,
 138,
 33,
 2938,
 67,
 56,
 41,
 25,
 15,
 105,
 1,
 6,
 3,
 3,
 2,
 23,
 1,
 1,
 1,
 1,
 2,
 5,
 269,
 1,
 6,
 1,
 13,
 61,
 4,
 99,
 80,
 6,
 219,
 4,
 97,
 5,
 1,
 2,
 1,
 1,
 56,
 13,
 1,
 19,
 4,
 183,
 1,
 1,
 1,
 1,
 3,
 75,
 55,
 7,
 19,
 5,
 8,
 9,
 32,
 68,
 12,
 48,
 2,
 12,
 2,
 3,
 24,
 3,
 39,
 5,
 1,
 46,
 1,
 8,
 9,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 3,
 13,
 5,
 1,
 1,
 5,
 1,
 50,
 33,
 1,
 1,
 6,
 6,
 1,
 1,
 1,
 25,
 1,
 614,
 1,
 8,
 1,
 3,
 1,
 7,
 1,
 5,
 1,
 5,
 1,
 4,
 55,
 1,
 17,
 1,
 5,
 1,
 5,
 1,
 3,
 1,
 13,
 1,
 3,
 1,
 3,
 1,
 3,
 1,
 9,
 16,
 2,
 25,
 63,
 1,
 17,
 30,
 1,
 1,
 1,
 1,
 9,
 1,
 5,
 9,
 1,
 24,
 14,
 5,
 2,
 2,
 4,
 1,
 1,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 58,
 21,
 1,
 3,
 22,
 6,
 13,
 23,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 23,
 1,
 1,
 3,
 1,
 1,
 49,
 1,
 1,
 2,
 1,

In [ ]:
blocks[0], inputs[0], outputs[0]

In [14]:
blocks_comb_subset = []
block_in_comb_subset = defaultdict(bool)
for block in netlist.get_block_graph():
    if block.type != 'dffeas':
        blocks_comb_subset.append(block)
        block_in_comb_subset[block] = True

In [15]:
net_graph = NetGraph(blocks_comb_subset)
netlist.net_graph()
net_blocks = netlist.get_net_graph()
blocks = netlist.get_block_graph()
for source_sinks in net_blocks.values():
    block_num = source_sinks['source']
    if block_num != []:
        source_num = list(block_num[0])[0]
        source = blocks[source_num]
        if block_in_comb_subset[source]:
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]
                if block_in_comb_subset[sink]:
                    net_graph.addEdge(source, sink)
                    net_graph.addEdge(sink, source)                    

In [16]:
net_graph.connectedComponents()

RecursionError: maximum recursion depth exceeded while calling a Python object